In [1]:
!pip install -q -U albumentations
!echo "$(pip freeze | grep albumentations) is successfully installed"

albumentations==1.3.0 is successfully installed


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install detecto --quiet

In [4]:
import cv2
from matplotlib import pyplot as plt
from detecto.utils import xml_to_csv
import albumentations as A
import os
from PIL import Image
from matplotlib import cm
import numpy as np

In [5]:
#Make a csv files out of the xml files in the directory
labels = xml_to_csv('/content/drive/MyDrive/Augmentation/Aaloo Chips')
#Save in labels.csv
labels.to_csv('labels.csv')

In [6]:
labels

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id
0,41E9EQShzJL.jpg,500,500,Aloo Chips,150,61,347,235,0
1,4-8-potato-chips-aalu-ke-chips-4-8-kg-jai-jine...,416,416,Aloo Chips,225,133,329,226,1
2,41-7hBikCOL.jpg,500,410,Aloo Chips,135,29,348,114,2
3,200-homemade-raw-potato-chips-potato-fryums-re...,312,271,Aloo Chips,53,137,132,211,3
4,200-home-made-raw-potato-chips-dry-potato-chip...,416,416,Aloo Chips,49,174,156,240,4


In [7]:
BOX_COLOR = (255, 0, 0)
TEXT_COLOR = (255, 255, 255) 

#Code to visualize the border boxes just for viewing sake
#Won't be needed while implementing 

def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    x_min, y_min, w, h,cla = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img

def visualize(image, bboxes, class_name):
    img = image.copy()
    for bbox in bboxes:
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [8]:
def Rotate():
  return A.Compose([A.ShiftScaleRotate(always_apply=True,p=0.5)],bbox_params=A.BboxParams(format='pascal_voc'))

def HFlip():
  return A.Compose([A.HorizontalFlip(always_apply=True,p=0.5)],bbox_params=A.BboxParams(format='pascal_voc'))

def Brightness():
  return A.Compose([A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=True, p=0.5)],bbox_params=A.BboxParams(format='pascal_voc'))

In [9]:
#Data augmentation code 
def data_augment(img_path,class_name,b_array,ch):
  image = cv2.imread(img_path)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  bboxes = [b_array]
  category = [class_name]
  if ch==1:
    transform = Rotate() #Use the rotation transform
    name_of_transform = " Rotated"
  elif ch==2:
    transform = HFlip() #Use the Horizontal flip transform
    name_of_transform = " Flipped"
  elif ch==3:
    transform = Brightness() #Use the random brightness transform
    name_of_transform = " Brightened"
  # A.Compose([
  #   #A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=True, p=0.5)
  #   #A.RandomGamma (gamma_limit=(80, 200), eps=None, always_apply=True, p=1)
  #   #A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=True, p=0.5)
  #   #A.HorizontalFlip(p=0.5)
  #   A.ShiftScaleRotate(always_apply=True,p=0.5)
  #   ], bbox_params=A.BboxParams(format='pascal_voc'))
  transformed = transform(image=image, bboxes=bboxes, class_labels=category) #Use the transform function
  transformed_image = transformed['image']
  transformed_bboxes = transformed['bboxes']
  transformed_class_labels = transformed['class_labels']
  return transformed_image,transformed_bboxes,transformed_class_labels,name_of_transform #Return the transformed images,bboxes and the name

In [10]:
base_path="/content/drive/MyDrive/Augmentation/Aaloo Chips/"
os.chdir(base_path) #Changing the directory to the base path

In [11]:
for i in range(0,len(labels)): 
  path = base_path+labels.iloc[i]['filename']
  class_name = labels.iloc[i]['class']
  #Getting bbox from the csv file
  b_arr = [labels.iloc[i]['xmin'],labels.iloc[i]['ymin'],labels.iloc[i]['xmax'],labels.iloc[i]['ymax'],labels.iloc[i]['class']] 
  for j in range(1,4):
    #Looping all the transforms to make new image for each transform 
    img,bbox,class_l,name = data_augment(path,class_name,b_arr,j)
    name_of_file = labels.iloc[i]['filename'] + name
    im = Image.fromarray(img)
    #Save the image in the drive
    im.save(f"{name_of_file}.jpg")
    width = im.width
    height = im.height
    xmin,ymin,xmax,ymax = bbox[0][0],bbox[0][1],bbox[0][2],bbox[0][3]
    #Inserting a new row in the labels.csv for the augmented images
    labels.loc[len(labels.index)] = [name_of_file,width,height,bbox[0][4],xmin,ymin,xmax,ymax,len(labels.index)] 
  #visualize(img, bbox,bbox[0][4])

In [12]:
labels

,filename,width,height,class,xmin,ymin,xmax,ymax,image_id
0,41E9EQShzJL.jpg,500,500,Aloo Chips,150.000000,61.000000,347.000000,235.000000,0
1,4-8-potato-chips-aalu-ke-chips-4-8-kg-jai-jine...,416,416,Aloo Chips,225.000000,133.000000,329.000000,226.000000,1
2,41-7hBikCOL.jpg,500,410,Aloo Chips,135.000000,29.000000,348.000000,114.000000,2
3,200-homemade-raw-potato-chips-potato-fryums-re...,312,271,Aloo Chips,53.000000,137.000000,132.000000,211.000000,3
4,200-home-made-raw-potato-chips-dry-potato-chip...,416,416,Aloo Chips,49.000000,174.000000,156.000000,240.000000,4
5,41E9EQShzJL.jpg Rotated,500,500,Aloo Chips,183.275919,34.811794,421.026392,258.627632,5
6,41E9EQShzJL.jpg Flipped,500,500,Aloo Chips,153.000000,61.000000,350.000000,235.000000,6
7,41E9EQShzJL.jpg Brightened,500,500,Aloo Chips,150.000000,61.000000,347.000000,235.000000,7
8,4-8-potato-chips-aalu-ke-chips-4-8-kg-jai-jine...,416,416,Aloo Chips,239.006162,120.274181,360.526275,231.790209,8
9,4-8-potato-chips-aalu-ke-chips-4-8-kg-jai-jine...,416,416,Aloo Chips,87.000000,133.000000,191.000000,226.000000,9
